In [28]:
import numpy as np 
import pandas as pd 
import random
from collections import Counter
from typing import Tuple
from typing import List

In [2]:
class colors():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['♠', '♦', '♥', '♣'] #['S','H','D','C'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [3]:
class numbers():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [11]:
class card():
    __slots__ = '__color', '__number'
    
    def __init__(self,aColor, aNumber):
        
        self.__color = aColor
        self.__number = aNumber
        
    def getColor(self):
        return self.__color
    
    def getNumber(self):
        return self.__number
    

In [104]:
class hand():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard:card):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            

        

In [105]:
class deck():
    __slots__ = '__cards'
        
    def __init__(self):
        self.__cards = []
        for color in colors().getPossibilities():
            for number in numbers().getPossibilities():
                self.__cards.append(card(color, number))
        random.shuffle(self.__cards)
    
    def drawCard(self):
        return self.__cards.pop(0)
    
    def getDeck(self):
        return self.__cards

In [106]:
class river():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard:card):
        self.__cards.append(aCard)
        self.__numbersOnly.append(aCard.getNumber())
        self.__colorsOnly.append(aCard.getColor())
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly
    
    def visualizeHand(self):
        myVisu = []
        for card in self.__cards:
            myVisu.append(card.getNumber() + card.getColor())
        print(myVisu)
            


In [129]:
def isFlush(aHand:hand) -> bool:
    return max(Counter(aHand.getColors()).values()) >= 5

def isStraight(aHand:hand) -> Tuple[bool, int]:
    first = True
    countConseq = 0
    maxConSeq = 0
    lastCardSeq = 0
    myIndexedSortedHand = sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])
    for idx in myIndexedSortedHand:
        if not first:
            if idx == myLastIdx+1:
                countConseq = countConseq+1
                lastCardSeq = myLastIdx #to compare between straights
            else:
                countConseq = 0
            if countConseq > maxConSeq :
                maxConSeq = countConseq
        first = False
        myLastIdx = idx
    
    return maxConSeq>=5,int(lastCardSeq)
    
    
def howManySameNumber(aHand:hand)-> Tuple[List,List,List]:
    myCount = Counter(aHand.getNumbers())
    
#     nbFourOfAKind = Counter(myCount.values())[4] #how many four of a kind ..
    cards4 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 4]]
#     nbThreeOfAKind = Counter(myCount.values())[3]
    cards3 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 3]]
#     nbPair = Counter(myCount.values())[2]
    cards2 = [numbers().getPossibilities().index(x) for x in [k for (k, v) in myCount.items() if v == 2]]
    
    return cards2,cards3,cards4
    
def singleCard(aHand:hand, idxBest=1): #idxBest starts at 1 for best, 2 for snd Best
    
    return sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])[-idxBest] #get number with highest

In [130]:
def getValue(aHand:hand) -> float:
    myValue = 0
    myStraight, myidx = isStraight(aHand)
    myFlush = isFlush(aHand)
    myRoyal = myidx is len(numbers().getPossibilities())-1
    
    myPairs, myThrees, myFour = howManySameNumber(aHand) 
    
    myValue = myValue + myStraight*myFlush*myRoyal*30000000000 + myStraight*myFlush*1500000000*myidx
   
    if len(myFour):
        myFourValue = 100000000*myFour[0]
    else:
        myFourValue = 0
    
    if len(myThrees)==2 or (len(myThrees)==1 and len(myPairs)):
        myFullHouse= 600000 * max(myThrees)
    else:
        myFullHouse = 0
    
    
    if len(myPairs)==2:
        myTwoPairs = 70000 * max(myPairs)
    else:
        myTwoPairs = 0
    
    if len(myThrees)==1:
        myThreeCard = 4000 * max(myThrees)
    else:
        myThreeCard = 0
        
    if len(myPairs)==1:
        myOnePair = 200 * max(myPairs)
    else:
        myOnePair = 0
    
    myValue = myValue + myFourValue + myFullHouse + 40000*myFlush + 2000*myStraight*myidx
    myValue = myValue + myThreeCard + myTwoPairs + myOnePair + singleCard(aHand,1)*10 + singleCard(aHand,2)*.5+singleCard(aHand,3)*.025
    
    return myValue
    

In [131]:
a = ['K', 'J', 'A','A'] 
possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A']

sorted(np.unique(a), key=lambda x: possibilities.index(x))

['J', 'K', 'A']

In [154]:
nbPlayers = 5
myDeck = deck()
mySmallHands = [] #individualHand
myBigHands = [] #Hand with River
for i in range(0,nbPlayers):
    mySmallHands.append(hand())
    myBigHands.append(hand())
myRiver = river()

In [155]:

#draw cards
for drawnCard in range(0,2):
    for player in range(0,nbPlayers):
        myCard = myDeck.drawCard()
        mySmallHands[player].addCard(myCard)
        myBigHands[player].addCard(myCard)
        
#draw cards in the river
for drawnCard in range(0,5):
    myCard = myDeck.drawCard()
    
    myRiver.addCard(myCard)
    for player in range(0,nbPlayers):
        myBigHands[player].addCard(myCard)

In [156]:
for i in range(0,nbPlayers):
    print(getValue(myBigHands[i]))

2105.15
2105.175
700105.175
2115.25
700105.15


In [157]:
for i in range(0,nbPlayers):
    print(myBigHands[i].visualizeHand())

['3♦', '4♦', '8♥', '6♠', '7♠', 'K♣', 'K♠']
None
['9♣', '5♠', '8♥', '6♠', '7♠', 'K♣', 'K♠']
None
['9♥', '8♠', '8♥', '6♠', '7♠', 'K♣', 'K♠']
None
['A♣', '5♣', '8♥', '6♠', '7♠', 'K♣', 'K♠']
None
['2♣', '7♣', '8♥', '6♠', '7♠', 'K♣', 'K♠']
None


['J♣', '3♣', 'A♥', '7♠', '7♥', '2♥', '5♥']


ValueError: <class 'int'> is not in list

In [95]:
aHand.getNumbers()

[int, 'A', 'Q', '8', 'Q', 'K', '4', '6']

In [115]:
aHand = myBigHands[1]
sorted(np.unique(aHand.getNumbers()), key=lambda x: numbers().getPossibilities().index(x))[-1]

'A'

In [129]:
Counter(aHand.getNumbers())

Counter({'2': 1, '3': 2, '6': 1, 'A': 1, 'K': 2})

In [138]:
sorted(np.unique(aHand.getNumbers()), key=lambda x: numbers().getPossibilities().index(x))

[0, 1, 1, 4, 10, 10, 11]

In [136]:
np.unique(aHand.getNumbers())

array(['2', '3', '6', 'A', 'K'], 
      dtype='<U1')

In [134]:
[numbers().getPossibilities().index(x) for x in aHand.getNumbers()]

[1, 11, 1, 10, 10, 0, 4]

In [141]:
sorted([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])[-1]

11

In [183]:
Counter(aHand.getNumbers())

Counter({'2': 1, '3': 2, '6': 1, 'A': 1, 'K': 2})

In [177]:
Counter(aHand.getNumbers()).keys()

dict_keys(['3', 'A', 'K', '2', '6'])

In [179]:
Counter(aHand.getNumbers()).values().index(2)

AttributeError: 'dict_values' object has no attribute 'index'

In [202]:
[numbers().getPossibilities().index(x) for x in aHand.getNumbers()]

[1, 11, 1, 10, 10, 0, 4]

In [204]:
Counter([numbers().getPossibilities().index(x) for x in aHand.getNumbers()])

Counter({0: 1, 1: 2, 4: 1, 10: 2, 11: 1})

In [212]:
shuh = [numbers().getPossibilities().index(x) for x in [k for (k, v) in Counter(aHand.getNumbers()).items() if v == 2]]

IndexError: list index out of range